# Trending Wikipedia articles with current news context

- Adding new prompt/column to get a very short news relation summary
- Save to Supabase


In [30]:

# !pip install openai supabase dotenv requests

In [31]:
from dotenv import load_dotenv
import os

load_dotenv()

SERPER_API_KEY = os.environ["SERPER_API_KEY"] 
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"] 
WIKIPEDIA_APP_NAME = os.environ["WIKIPEDIA_APP_NAME"] 

### Set date to query

In [32]:
# Python 3

# Get today's date in YYYY/MM/DD format.
import datetime

# today = datetime.datetime.now() - datetime.timedelta(days=1)
today = datetime.datetime.now()

# date_to_query = today - datetime.timedelta(days=7)
date_to_query = today



In [33]:
import os
from supabase import create_client

supabase_url = 'https://htmlbdzwdcxmnxuogycv.supabase.co'
supabase_key = os.environ.get('SUPABASE_KEY')
supabase = create_client(supabase_url, supabase_key)

In [51]:

    
current_agent_version = supabase.table('agent_versions') \
    .select('*') \
    .lte('effective_date', today) \
    .order('effective_date', desc=True) \
    .limit(1) \
    .execute()

print(current_agent_version.data[0]["name"])
print(current_agent_version.data[0]["step_5_prompt_text"])
print(current_agent_version)


Version 11
Given the trending reason and extract below, generate a summary that does not mention or reference the article's title. Focus only on describing its content. For instance, instead of "Justin Rose is igniting interest with a record-breaking opening round at the Masters that earned him a three-shot lead. His stellar performance is stirring excitement among golf fans, reviving memories of his impressive career milestones. A mix of athletic brilliance and a longstanding reputation is making his play a hot topic today." produce "Justin Rose is leading the Masters tournament with a 3 shot lead." If the article is about an event, deliberately state the date of the event. For instance if the article is about Passover be sure to explain "Passover trending because it is between [x date] and [x date]" 
If the article is about a person and the extract refernces dates from past years, do not directly reference the dates. 
The summary should be between 8 and 15 words (shorter is preferabl

### Get feed data from date specified

create an account here: https://api.wikimedia.org/wiki/Special:CreateAccount

In [35]:
import requests
import json

language_code = 'en' # English
headers = {
  #'Authorization': 'Bearer YOUR_ACCESS_TOKEN',
  'User-Agent': f"""YOUR_APP_NAME ({WIKIPEDIA_APP_NAME})"""
}

base_url = 'https://api.wikimedia.org/feed/v1/wikipedia/'
url = base_url + language_code + '/featured/' + date_to_query.strftime('%Y/%m/%d')
print(url)
response = requests.get(url, headers=headers)

featured_feed = response.json()
feed_date_str = featured_feed["mostread"]["date"].rstrip("Z")
print(f"Feed date string: {feed_date_str}")
feed_date = datetime.datetime.strptime(feed_date_str, '%Y-%m-%d')
print(f"Feed date: {feed_date}")
feed_date_long =  feed_date.strftime('%B %d, %Y')
print(f"Feed date long: {feed_date_long}")
print(f"API call: {url}")

https://api.wikimedia.org/feed/v1/wikipedia/en/featured/2025/04/14
Feed date string: 2025-04-13
Feed date: 2025-04-13 00:00:00
Feed date long: April 13, 2025
API call: https://api.wikimedia.org/feed/v1/wikipedia/en/featured/2025/04/14


### Build data structure with all relevant information and placeholders for LLM responses

In [36]:
article_list = []
rank_counter = 1 

for item in featured_feed['mostread']['articles']:
    title = item['title']
    normalized_title = item['titles']['normalized']
    view_count = item['views']
    link = item['content_urls']['desktop']['page']
    extract = item['extract_html']
    thumbnail = item.get('thumbnail', {}).get('source', None)
    view_history = item['view_history']
    mystery_rank = item['rank'] # Not sure why the first article is always ranked 3

    article={
        'date': feed_date_str,
        'title': title,
        'normalized_title': normalized_title,
        'summary': '',
        'view_count': view_count,
        'rank': rank_counter,
        'mystery_rank': mystery_rank,
        'link': link,
        'thumbnail': thumbnail,
        'extract': extract,
        'raw_text': '',
        'trending_reason': '',
        'view_history': view_history,
        'is_newly_trending': '',
        'view_delta_percentage': '',
        'raw_news_results': '',
        'news_relation': '',
        'news_relation_short': '',
        'categories': '',
        'news_relation_json': ''
    }

    article_list.append(article)
    rank_counter += 1

## Determine if the article is newly trending. If it is, add to new list

- Filter list to only articles with meaningful spike in views



In [37]:
newly_trending_article_list = []

for article in article_list:
    view_history = article['view_history']

    view_history_length = len(view_history)

    yesterdays_views = view_history[view_history_length-2]['views']
    todays_views = view_history[view_history_length-1]['views']

    # Handle division by zero case
    if yesterdays_views == 0:
        view_delta_percentage = float('inf') if todays_views > 0 else 0
    else:
        view_delta_percentage = ((todays_views - yesterdays_views) / yesterdays_views) * 100

    # More concise assignment using a boolean expression
    article['is_newly_trending'] = view_delta_percentage > 100
    article['view_delta_percentage'] = int(view_delta_percentage)
    print(f"{article['normalized_title']} view delta: {article['view_delta_percentage']} is newly trending: {article['is_newly_trending']}")

  

Rory McIlroy view delta: 404 is newly trending: True
Justin Rose view delta: 522 is newly trending: True
Bryson DeChambeau view delta: 115 is newly trending: True
List of Masters Tournament champions view delta: 158 is newly trending: True
Grand Slam (golf) view delta: 809 is newly trending: True
UFC 314 view delta: 295 is newly trending: True
Kyren Lacy view delta: 73984 is newly trending: True
Paddy Pimblett view delta: 497 is newly trending: True
Jaat (film) view delta: 5 is newly trending: False
1989 Tiananmen Square protests and massacre view delta: 242 is newly trending: True
Palm Sunday view delta: 511 is newly trending: True
A Minecraft Movie view delta: 3 is newly trending: False
List of Black Mirror episodes view delta: 15 is newly trending: False
Alexander Volkanovski view delta: 592 is newly trending: True
ChatGPT view delta: -6 is newly trending: False
Tiger Woods view delta: 202 is newly trending: True
Masters Tournament view delta: 143 is newly trending: True
Mickey Rour

In [38]:
# newlyTrendingCount = article_list.filter(article => article.is_newly_trending === true).length;
# newly_trending_count = article_list[article_list['is_newly_trending'] == True].shape[0]
newly_trending_count = sum(1 for article in article_list if article.get('is_newly_trending') == True)
print( f"Total trending: {len(article_list)}")
print( f"Newly trending: {newly_trending_count}" )

Total trending: 47
Newly trending: 29


### Get first 5000 characters of article

In [39]:
for article in article_list:
  # Download raw text of article
  if article.get('is_newly_trending') == True:
    url = f"https://en.wikipedia.org/w/index.php?title={article['title']}&action=raw"
    print(url)
    
    article_text = requests.get(url).text
    article_text_truncated = article_text[:5000]
    article['raw_text'] = article_text_truncated

https://en.wikipedia.org/w/index.php?title=Rory_McIlroy&action=raw
https://en.wikipedia.org/w/index.php?title=Justin_Rose&action=raw
https://en.wikipedia.org/w/index.php?title=Bryson_DeChambeau&action=raw
https://en.wikipedia.org/w/index.php?title=List_of_Masters_Tournament_champions&action=raw
https://en.wikipedia.org/w/index.php?title=Grand_Slam_(golf)&action=raw
https://en.wikipedia.org/w/index.php?title=UFC_314&action=raw
https://en.wikipedia.org/w/index.php?title=Kyren_Lacy&action=raw
https://en.wikipedia.org/w/index.php?title=Paddy_Pimblett&action=raw
https://en.wikipedia.org/w/index.php?title=1989_Tiananmen_Square_protests_and_massacre&action=raw
https://en.wikipedia.org/w/index.php?title=Palm_Sunday&action=raw
https://en.wikipedia.org/w/index.php?title=Alexander_Volkanovski&action=raw
https://en.wikipedia.org/w/index.php?title=Tiger_Woods&action=raw
https://en.wikipedia.org/w/index.php?title=Masters_Tournament&action=raw
https://en.wikipedia.org/w/index.php?title=Nicky_Katt&act

In [40]:
from openai import OpenAI
openAIClient = OpenAI(api_key=OPENAI_API_KEY)
    

In [41]:

openAIClient = OpenAI(api_key=OPENAI_API_KEY)
step_1_prompt_text = current_agent_version.data[0]["step_1_prompt_text"]
step_1_model_name = current_agent_version.data[0]["step_1_model_name"]
# step_1_prompt_text = f"""Given the Wikipedia extract below, generate a summary that does not mention or reference the article's title. Focus only on describing its content. For instance, instead of "Disney's Snow White is a live-action reimagining of the classic fairy tale," produce "Disney's live-action reimagining of the classic fairy tale. . The summary should be between 8 and 15 words (shorter is preferable).
# Title: {title}
# Extract: {extract}"""

for article in article_list:

    extract = article['extract']
    title = article['normalized_title']

    print(f"Analyzing {title}")

    summary_prompt = step_1_prompt_text.replace("{extract}", extract).replace("{title}", title)
    print(summary_prompt)

    openAISummary = openAIClient.chat.completions.create(
        model=step_1_model_name,
        messages=[
            {
            "role": "user",
            "content": summary_prompt
            }
        ],
        temperature=0.3,
        max_tokens=128,
        top_p=1
    )

    openAISummaryResponse = openAISummary.choices[0].message.content

    print(f"""openAISummaryResponse: {openAISummaryResponse}""")

    article['summary'] =  openAISummaryResponse



    

Analyzing Rory McIlroy
Given the Wikipedia extract below, generate a summary that does not mention or reference the article's title. Focus only on describing its content. For instance, instead of "Disney's Snow White is a live-action reimagining of the classic fairy tale," produce "Disney's live-action reimagining of the classic fairy tale." If there is an actor, instead of being vague like "An American actor known for roles in television and film." produce "An American actor known for roles in Euphoria and Grey's Anatomy". Make sure there is basic context included instead of "A retired gangster returns to violence to save his wrongly arrested son." generate "An Indian movie about a retired gangster returns to violence to save his wrongly arrested son." The summary should be between 8 and 15 words (shorter is preferable).
Title: Rory McIlroy
Extract: <p><b>Rory Daniel McIlroy</b> is a Northern Irish professional golfer who plays on the European Tour and the PGA Tour. He is a former wor

In [42]:
# import requests
# import json

url = "https://google.serper.dev/news"

for article in article_list:
  if article.get('is_newly_trending') == True:
    title = article['normalized_title']
    payload = json.dumps({
      "q": title,
      "autocorrect": False,
      "tbs": "qdr:w"
    })
    headers = {
      'X-API-KEY': SERPER_API_KEY,
      'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    article['raw_news_results'] = response.json()

    print(response.text)

{"searchParameters":{"q":"Rory McIlroy","type":"news","autocorrect":false,"tbs":"qdr:w","engine":"google"},"news":[{"title":"Rory coaster ride nets Masters win, career Slam","link":"https://www.espn.com/golf/story/_/id/44662310/rory-mcilroy-wins-masters-playoff-earn-career-grand-slam","snippet":"AUGUSTA, Ga. -- Rory McIlroy's long, painful wait for the career Grand Slam is finally over. And the greatest achievement of his career was as nerve-racking...","date":"15 hours ago","source":"ESPN","imageUrl":"https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSPwP2Ku1HPy00MQ7TLDVspucx73SfgsXHG8EukwxMtLGAtYTV4wTnTPuNp0Q&s","position":1},{"title":"This is why so many people want to see Rory McIlroy win The Masters","link":"https://www.sbnation.com/golf/2025/4/13/24407209/why-golf-fans-want-rory-mcilroy-win-masters","snippet":"There are so many people rooting for Rory McIlroy to win The Masters... here is why.","date":"1 day ago","source":"SBNation.com","imageUrl":"https://encrypted-tbn0.gsta

In [43]:

llm_miss_response = "No relevant news found"

step_3_prompt_text = current_agent_version.data[0]["step_3_prompt_text"]
step_3_model_name = current_agent_version.data[0]["step_3_model_name"]


for article in article_list:
    if article.get('is_newly_trending') == True:

        title = article['normalized_title']
        news = article['raw_news_results']['news']
        extract = article['extract']

        print(f"Analyzing {title}")
        if len(news) == 0:
            print(f"No news results for {title}")
            continue

        # If news is already a JSON string but needs reformatting
        news_str = json.dumps(news, indent=2)
        # Now use the string version in your replacement
        news_prompt = step_3_prompt_text.replace("{title}", title).replace("{news}", news_str).replace("{extract}", extract)

        openAINewsRelation = openAIClient.chat.completions.create(
            model=step_3_model_name,
            messages=[
                {
                "role": "user",
                "content": news_prompt
                }
            ],
            max_completion_tokens=2048
        )

        openAINewsResponse = openAINewsRelation.choices[0].message.content

        print(openAINewsResponse)

        article['news_relation'] =  openAINewsResponse


    

Analyzing Rory McIlroy
Rory clinched his long-awaited career Grand Slam with a dramatic playoff win at the Masters. His victory represents years of near misses and high-stakes moments that have captivated golf fans. The blend of history-making accomplishment and last-minute triumph has sparked lively conversations among enthusiasts.
Analyzing Justin Rose
Justin Rose has caught editors’ attention with his dramatic performances at the Masters, highlighted by his near wins and ability to remain competitive despite setbacks. Headlines underscore his repeated near-misses and the narrative of a seasoned veteran still pushing for glory. His storied career—including historic wins, runner-up finishes, and overcoming injuries—adds an extra layer of intrigue.
Analyzing Bryson DeChambeau
Bryson’s high-stakes Masters performance and offbeat equipment tests are striking a chord. Headlines celebrate his daring tweaks and last-minute heroics on the course. His unique journey—from PGA triumphs to the L

In [ ]:

llm_miss_response = "No relevant news found"

step_5_prompt_text = current_agent_version.data[0]["step_5_prompt_text"]
step_5_model_name = current_agent_version.data[0]["step_5_model_name"]

# step_5_prompt_text = """
# Given the trending reason below, generate a summary that does not mention or reference the article's title. Focus only on describing its content. For instance, instead of "Justin Rose is igniting interest with a record-breaking opening round at the Masters that earned him a three-shot lead. His stellar performance is stirring excitement among golf fans, reviving memories of his impressive career milestones. A mix of athletic brilliance and a longstanding reputation is making his play a hot topic today." produce "Justin Rose is leading the Masters tournament with a 3 shot lead." . The summary should be between 8 and 15 words (shorter is preferable).
# Trending Reason: {news_relation}
# """
# step_5_model_name = "gpt-4o-mini"

for article in article_list:
    if article.get('is_newly_trending') == True:
        news_relation = article['news_relation']
        extract = article['extract']

        print(f"Analyzing {article['title']}")
        # if len(news) == 0:
        #     print(f"No news results for {title}")
        #     continue

        # If news is already a JSON string but needs reformatting
        news_str = json.dumps(news, indent=2)
        # Now use the string version in your replacement
        short_news_prompt = step_5_prompt_text.replace("{news_relation}", news_relation).replace("{extract}", extract)

        openAINewsRelationShort = openAIClient.chat.completions.create(
            model=step_5_model_name,
            messages=[
                {
                "role": "user",
                "content": short_news_prompt
                }
            ],
            max_completion_tokens=2048
        )

        openAIShortNewsResponse = openAINewsRelationShort.choices[0].message.content

        print(f"News relation short: {openAIShortNewsResponse}")

        article['news_relation_short'] =  openAIShortNewsResponse


    

Analyzing Rory_McIlroy
step_5_prompt_text: Given the trending reason and extract below, generate a summary that does not mention or reference the article's title. Focus only on describing its content. For instance, instead of "Justin Rose is igniting interest with a record-breaking opening round at the Masters that earned him a three-shot lead. His stellar performance is stirring excitement among golf fans, reviving memories of his impressive career milestones. A mix of athletic brilliance and a longstanding reputation is making his play a hot topic today." produce "Justin Rose is leading the Masters tournament with a 3 shot lead." If the article is about an event, deliberately state the date of the event. For instance if the article is about Passover be sure to explain "Passover trending because it is between [x date] and [x date]" 
If the article is about a person and the extract refernces dates from past years, do not directly reference the dates. 
The summary should be between 8 an

In [45]:
step_4_prompt_text = current_agent_version.data[0]["step_4_prompt_text"]
step_4_model_name = current_agent_version.data[0]["step_4_model_name"]


for article in article_list:

    extract = article['extract']
    news_relation = article['news_relation']
    title = article['title']

    print(f"Analyzing {title}")

    category_prompt = step_4_prompt_text.replace("{extract}", extract).replace("{news_relation}", news_relation)

    openAICategories = openAIClient.chat.completions.create(
        model=step_4_model_name,
        messages=[
            {
            "role": "user",
            "content": category_prompt
            }
        ],
        temperature=0.3,
        max_tokens=128,
        top_p=1
    )

    openAICategoriesResponse = openAICategories.choices[0].message.content

    print(openAICategoriesResponse)

    article['categories'] =  openAICategoriesResponse

Analyzing Rory_McIlroy
```json
["Sports", "News"]
```
Analyzing Justin_Rose
```json
["Sports", "News"]
```
Analyzing Bryson_DeChambeau
```json
["Sports", "News", "Celebrities"]
```
Analyzing List_of_Masters_Tournament_champions
```json
["Sports", "History", "News"]
```
Analyzing Grand_Slam_(golf)
```json
["Sports", "News", "History"]
```
Analyzing UFC_314
```json
["Sports", "News"]
```
Analyzing Kyren_Lacy
```json
["Sports", "Death", "Crime", "News"]
```
Analyzing Paddy_Pimblett
```json
["Sports", "News", "Celebrities"]
```
Analyzing Jaat_(film)
```json
["Movies", "Celebrities"]
```
Analyzing 1989_Tiananmen_Square_protests_and_massacre
```json
["Politics", "History", "News", "Military"]
```
Analyzing Palm_Sunday
```json
["History", "News"]
```
Analyzing A_Minecraft_Movie
```json
["Movies", "Video Games", "Celebrities"]
```
Analyzing List_of_Black_Mirror_episodes
```json
["Television", "Technology", "Movies"]
```
Analyzing Alexander_Volkanovski
```json
["Sports", "News", "Celebrities"]


#### Build HTML Page to display results

In [46]:
# Start building the HTML
html_title = f"<h1>Newly Trending on {feed_date_long}</h1>\n"
if len(article_list) > 0:
    html_list = "<ol>\n"

    # Iterate through the data
    for item in article_list:
        title = item['normalized_title']
        link = item['link']
        view_count = item['view_count']
        
        thumbnail = item['thumbnail']
        trending_reason = item['trending_reason']
        news_relation = item['news_relation']
        news_relation_short = item['news_relation_short']
        
        extract = item['extract']
        summary = item['summary']

        # Handle null thumbnail
        if thumbnail:
            thumbnail_html = f'<img src="{thumbnail}" alt="Thumbnail for {title}"/><br>'
        else:
            thumbnail_html = ''
        
        news_relation_output = f"<strong>News related to this:</strong> {news_relation}<br><br>"

        short_news_relation_output = f"<strong>SHORT News related to this:</strong> {news_relation_short}<br><br>"

        view_history_list = "<ul>"
        for view in item['view_history']:
            view_history_list += f"<li><strong>{view['date'].split("Z")[0]}:</strong> {view['views']:,}</li>"
        view_history_list += "</ul>"

        # Create a list item for each entry
        if item['is_newly_trending'] == True:
            html_list += f"""
            <li>
                <h2>
                <a href="{link}" target="_blank">{title}</a><br>
                </h2>
                {thumbnail_html}
                <strong>Views:</strong><br>
                {view_history_list}<br><br>
                <strong>view_delta_percentage: </strong> {item['view_delta_percentage']}<br>
                <strong>Extract:</strong {extract}<br>
                <strong>Summary:</strong> {summary}<br>

                <h3>GPT o3 mini</h3>
                <strong>Reason for Trending:</strong> {trending_reason}<br><br>
                {news_relation_output}<br>
                {short_news_relation_output}<br>
                <strong>categories: </strong> {item['categories']}<br>
                
            </li>
            """
        else:
            html_list += f"""
            <li>
                <h2>
                <a href="{link}" target="_blank">{title}</a><br>
                </h2>                
                <strong>Views:</strong><br>
                {view_history_list}<br><br>
                <strong>view_delta_percentage: </strong> {item['view_delta_percentage']}<br>
                <strong>categories: </strong> {item['categories']}<br>
                
            </li>
            """
        

    # Close the HTML list
    html_list += "\n</ol>"
else:
    html_list = "<p>No articles are trending today.</p>"
html_page = html_title + html_list



### Display generated html

In [47]:
# # Display the HTML in the notebook (assuming Jupyter or similar)
# from IPython.display import display, HTML
# display(HTML(html_page))

In [48]:
print(current_agent_version.data[0]["id"])
for article in article_list:
  article['agent_version_id'] = '59acfe2e-6d54-46e1-b4cb-73f7bbed693b'
  print(f"{article['agent_version_id']} {article['date']} {article['title']} [{article['news_relation']}]")

712e49c6-2f9b-4bfa-be9a-4fc3cbecaf4d
712e49c6-2f9b-4bfa-be9a-4fc3cbecaf4d 2025-04-13 Rory_McIlroy [Rory clinched his long-awaited career Grand Slam with a dramatic playoff win at the Masters. His victory represents years of near misses and high-stakes moments that have captivated golf fans. The blend of history-making accomplishment and last-minute triumph has sparked lively conversations among enthusiasts.]
712e49c6-2f9b-4bfa-be9a-4fc3cbecaf4d 2025-04-13 Justin_Rose [Justin Rose has caught editors’ attention with his dramatic performances at the Masters, highlighted by his near wins and ability to remain competitive despite setbacks. Headlines underscore his repeated near-misses and the narrative of a seasoned veteran still pushing for glory. His storied career—including historic wins, runner-up finishes, and overcoming injuries—adds an extra layer of intrigue.]
712e49c6-2f9b-4bfa-be9a-4fc3cbecaf4d 2025-04-13 Bryson_DeChambeau [Bryson’s high-stakes Masters performance and offbeat equi

In [49]:
agent_version_id = current_agent_version.data[0]["id"]
for article in article_list:
    # Insert each article
    data = {
        'agent_version_id': agent_version_id,
        'trending_date': article['date'],
        'title': article['title'],
        'normalized_title': article['normalized_title'],
        'link': article['link'],
        'thumbnail': article['thumbnail'],
        'extract': article['extract'],
        'is_newly_trending': article['is_newly_trending'],
        'view_delta_percentage': article['view_delta_percentage'],
        'summary': article['summary'],
        'view_count': article['view_count'],
        'rank': article['rank'],
        'mystery_rank': article['mystery_rank'],
        'view_history': article['view_history'],
        'trending_reason': article['trending_reason'],
        'raw_news_results': article['raw_news_results'],
        'news_relation': article["news_relation"],
        'news_relation_short': article["news_relation_short"],
        'categories': article["categories"]
    }
    

    try:
        result = supabase.table('trending_articles').insert(data).execute()
        print(f"Inserted {article['title']}")
    except Exception as e:
        print(f"Error inserting {article['title']}: {str(e)}")
    
    # Optional: print success message

print("All data inserted successfully")

Inserted Rory_McIlroy
Inserted Justin_Rose
Inserted Bryson_DeChambeau
Inserted List_of_Masters_Tournament_champions
Inserted Grand_Slam_(golf)
Inserted UFC_314
Inserted Kyren_Lacy
Inserted Paddy_Pimblett
Inserted Jaat_(film)
Inserted 1989_Tiananmen_Square_protests_and_massacre
Inserted Palm_Sunday
Inserted A_Minecraft_Movie
Inserted List_of_Black_Mirror_episodes
Inserted Alexander_Volkanovski
Inserted ChatGPT
Inserted Tiger_Woods
Inserted Masters_Tournament
Inserted Mickey_Rourke
Inserted Nicky_Katt
Inserted Black_Mirror
Inserted Natalia_Grace
Inserted Good_Bad_Ugly
Inserted Michael_Chandler
Inserted Deaths_in_2025
Inserted Diego_Lopes_(fighter)
Inserted Scottie_Scheffler
Inserted Ludvig_Åberg
Inserted Karun_Nair
Inserted Jack_Nicklaus
Inserted List_of_men's_major_championships_winning_golfers
Inserted The_Amateur_(2025_film)
Inserted Terrorism_Confinement_Center
Inserted Benson_Boone
Inserted Josh_Shapiro
Inserted The_Last_of_Us_season_2
Inserted Sambhaji
Inserted Bryce_Mitchell
Inser